In [29]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn import MSELoss, CrossEntropyLoss
from torchvision import transforms
import torch.optim as optim
import os
import time

from data_loader import FaceDataset
from model import AgeGenderModel

In [31]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])  

full_dataset = FaceDataset(root_dir='half_dataset', transform=transform)

In [32]:
filtered_indices = [i for i, (data, label) in enumerate(full_dataset) if data is not None]
dataset = torch.utils.data.Subset(full_dataset, filtered_indices)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


In [33]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 4)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False, num_workers = 4)

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = AgeGenderModel().to(device)

age_criterion = MSELoss()
gender_criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader :
        images = images.to(device)
        ages = labels[:, 0].to(device)
        genders = labels[:, 1].long().to(device)

        optimizer.zero_grad()

        age_pred, gender_pred = model(images)

        age_loss = age_criterion(age_pred.squeeze(), ages)
        gender_loss = gender_criterion(gender_pred, genders)
        total_loss = age_loss + gender_loss

        total_loss.backward()

        optimizer.step()

        running_loss += total_loss.item()
    end_time = time.time()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):4f}, Time: {end_time - start_time:.2f}sec")

torch.save(model.state_dict(),'models/age_gender_model.path')
print("Training complete. Model saved.")


In [26]:
import torch
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, mean_absolute_error, confusion_matrix
import numpy as np

from data_loader import FaceDataset
from model import AgeGenderModel

In [ ]:
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [28]:
root_dir = '/Users/vaishnavishinde/Desktop/age_detector/dataset'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
model = AgeGenderModel().to(device)
model.load_state_dict(torch.load('age_gender_model.path'))
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'age_gender_model.path'

In [ ]:
full_dataset = UTKFaceDataset(root_dir=root_dir, transform=transform)
filtered_indices = [i for i, (data, label) in enumerate(full_dataset) if data is not None]
dataset = torch.utils.data.Subset(full_dataset, filtered_indices)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
_, _, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [ ]:


def main():
    root_dir = 'path/to/UTKFace'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Load the trained model
    model = AgeGenderModel().to(device)
    model.load_state_dict(torch.load('age_gender_model.pth'))
    model.eval()
    
    # Recreate the dataset split for a consistent test set
    full_dataset = UTKFaceDataset(root_dir=root_dir, transform=get_transforms())
    filtered_indices = [i for i, (data, label) in enumerate(full_dataset) if data is not None]
    dataset = torch.utils.data.Subset(full_dataset, filtered_indices)
    
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    _, _, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    all_ages = []
    all_age_preds = []
    all_genders = []
    all_gender_preds = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            ages = labels[:, 0].numpy()
            genders = labels[:, 1].numpy()
            
            age_pred, gender_pred = model(images)
            
            all_ages.extend(ages)
            all_genders.extend(genders)
            all_age_preds.extend(age_pred.squeeze().cpu().numpy())
            all_gender_preds.extend(torch.argmax(gender_pred, dim=1).cpu().numpy())

    # Gender metrics
    gender_accuracy = accuracy_score(all_genders, all_gender_preds)
    conf_matrix = confusion_matrix(all_genders, all_gender_preds)

    # Age metrics
    mae = mean_absolute_error(all_ages, all_age_preds)
    
    print(f"Gender Accuracy: {gender_accuracy:.4f}")
    print("Gender Confusion Matrix:\n", conf_matrix)
    print(f"Age Mean Absolute Error (MAE): {mae:.2f}")

if __name__ == '__main__':
    main()